In [1]:
import sqlite3 as sq3
import pandas as pd
import numpy as np

In [2]:
con = sq3.connect("stores.db")

*CÂU 2: Chúng tôi nên đặt hàng nhiều hơn hay ít hơn những sản phẩm nào?*

- *Tính tỷ lệ tăng trưởng đơn hàng (OGR - Ordered Growth Rate) của nửa đầu năm 2005 so với cùng kỳ 2004.*
  - *We calculate the OGR of the first half in 2005 and the same period in 2004.*
- *Lấy tỷ lệ tăng trưởng đơn hàng nhân với lượng đặt hàng của nửa cuối năm 2004 để ra được sản lượng đơn hàng dự kiến nửa cuối năm 2005.*
  - *We use above OGR to multiple with the quantityOrdered in the last half in 2004 => forecast the quantityOrdered for a last half 2005.*
- *Sau đó trừ đi cho lượng hàng tồn kho thì sẽ tính được lượng hàng cần đặt.*
  - *Reduce quantityOrdered for the last half 2005 with the in_stock => the quantity need to buy.*

In [21]:
sql_q2 = """WITH
first_half_2004_table AS (
SELECT p.productName, SUM(od.quantityOrdered) AS total_ordered, p.quantityInStock
  FROM orders AS o
  JOIN orderdetails AS od
    ON o.orderNumber = od.orderNumber
  JOIN products AS p
    ON od.productCode = p.productCode
 WHERE (o.orderDate BETWEEN '2004-01-01' AND '2004-05-31') AND (o.status NOT LIKE 'Cancelled')
 GROUP BY p.productName
 ORDER BY SUM(od.quantityOrdered) DESC
),

second_half_2004_table AS (
SELECT p.productName, SUM(od.quantityOrdered) AS total_ordered, p.quantityInStock
  FROM orders AS o
  JOIN orderdetails AS od
    ON o.orderNumber = od.orderNumber
  JOIN products AS p
    ON od.productCode = p.productCode
 WHERE (o.orderDate BETWEEN '2004-06-01' AND '2004-12-31') AND (o.status NOT LIKE 'Cancelled')
 GROUP BY p.productName
 ORDER BY SUM(od.quantityOrdered) DESC
),

first_half_2005_table AS (
SELECT p.productName, SUM(od.quantityOrdered) AS total_ordered, p.quantityInStock
  FROM orders AS o
  JOIN orderdetails AS od
    ON o.orderNumber = od.orderNumber
  JOIN products AS p
    ON od.productCode = p.productCode
 WHERE (o.orderDate BETWEEN '2005-01-01' AND '2005-05-31') AND (o.status NOT LIKE 'Cancelled')
 GROUP BY p.productName
 ORDER BY SUM(od.quantityOrdered) DESC
)


SELECT y4_2.productName, ROUND((y5_1.total_ordered *1.0 / y4_1.total_ordered) -1, 2) AS OGR_first_2005, 
	   (CASE
		  WHEN y4_2.total_ordered * ROUND((y5_1.total_ordered *1.0 / y4_1.total_ordered) -1,3) > 0 THEN CAST(y4_2.total_ordered*ROUND((y5_1.total_ordered *1.0 / y4_1.total_ordered) -1,3) AS INTEGER)
		  ELSE 0
	     END) AS order_last_half_2005,
	   y5_1.quantityInStock,
	   (CASE
		  WHEN y5_1.quantityInStock - CAST(y4_2.total_ordered*ROUND((y5_1.total_ordered *1.0 / y4_1.total_ordered) -1,3) AS INTEGER) <= 0 THEN ABS(y5_1.quantityInStock - CAST(y4_2.total_ordered*ROUND((y5_1.total_ordered *1.0 / y4_1.total_ordered) -1,3) AS INTEGER))
		  ELSE 0
	     END) AS order_requirement,
	   (CASE
	      WHEN y5_1.quantityInStock - CAST(y4_2.total_ordered*ROUND((y5_1.total_ordered *1.0 / y4_1.total_ordered) -1,3) AS INTEGER) >  0 THEN 'No'
		  ELSE 'Yes'
	     END) AS suggestion
 FROM first_half_2004_table AS y4_1
 JOIN first_half_2005_table AS y5_1
   ON y4_1.productName = y5_1.productName
 JOIN second_half_2004_table AS y4_2
   ON y4_1.productName = y4_2.productName
ORDER BY order_requirement DESC;"""

qry = con.execute(sql_q2).fetchall()
answer2 = pd.DataFrame(qry)
answer2 = answer2.rename(columns={0:'Product Name', 1:'OGR first 2005', 2:'Order for last half 2005', 3:'Quantity In Stock', 4:'Order Requirement', 5:'Suggestion'})
answer2

,Product Name,OGR first 2005,Order for last half 2005,Quantity In Stock,Order Requirement,Suggestion
0,1960 BSA Gold Star DBD34,0.78,257,15,242,Yes
1,1968 Ford Mustang,0.88,259,68,191,Yes
2,1997 BMW F650 ST,0.85,283,178,105,Yes
3,1992 Ferrari 360 Spider red,0.56,296,8347,0,No
4,1980s Black Hawk Helicopter,-0.18,0,5330,0,No
...,...,...,...,...,...,...
104,1980’s GM Manhattan Express,0.89,297,5099,0,No
105,1911 Ford Town Car,0.83,227,540,0,No
106,1903 Ford Model A,0.59,180,3913,0,No
107,1969 Dodge Charger,2.63,688,7323,0,No


In [23]:
orders = pd.read_sql("SELECT orderNumber, orderDate, status, customerNumber, shippedDate \
                FROM orders",con)
orderdetails = pd.read_sql("SELECT orderNumber, productCode, quantityOrdered, priceEach \
                FROM orderdetails",con)
products = pd.read_sql("SELECT productCode, productName, productLine, quantityInStock, buyPrice, MSRP \
                FROM products",con)
df = pd.merge(products, orderdetails, on='productCode')
datamix = pd.merge(df, orders, on='orderNumber')
first_half_2004 = datamix[(datamix['orderDate'] >= '2004-01-01') & (datamix['orderDate'] < '2004-06-01') & (datamix['status'] != 'Cancelled')].groupby(['productCode','productLine','productName','quantityInStock']).agg({'quantityOrdered': 'sum'}).reset_index()
last_half_2004 = datamix[(datamix['orderDate'] >= '2004-06-01') & (datamix['orderDate'] < '2004-12-31') & (datamix['status'] != 'Cancelled')].groupby(['productCode','productLine','productName','quantityInStock']).agg({'quantityOrdered': 'sum'}).reset_index()
first_half_2005 = datamix[(datamix['orderDate'] >= '2005-01-01') & (datamix['orderDate'] < '2005-06-01') & (datamix['status'] != 'Cancelled')].groupby(['productCode','productLine','productName','quantityInStock']).agg({'quantityOrdered': 'sum'}).reset_index()
## Tỷ lệ tăng trưởng đơn hàng của nửa đầu năm 2005 so với cùng kỳ 2004
OGR_first_2005 = (first_half_2005['quantityOrdered'] - first_half_2004['quantityOrdered']) / first_half_2004['quantityOrdered']
## Luợng đặt hàng nửa cuối năm 2005:
last_half_2005 = last_half_2004['quantityOrdered'] * OGR_first_2005
## Số lượng hàng cần đặt thêm / bớt
data_2005 = last_half_2004[['productCode', 'productLine', 'productName', 'quantityInStock']].copy()
data_2005['order_last_half_2005'] = last_half_2005
data_2005['order_requirement'] = round(data_2005['order_last_half_2005'] - data_2005['quantityInStock'], 0)
data_2005.loc[data_2005['order_requirement'] < 0, 'order_requirement'] = 0
data_2005.loc[data_2005['order_requirement'] == 0, 'suggestion'] = 'No'
data_2005.loc[data_2005['order_requirement'] > 0, 'suggestion'] = 'Yes'

data_2005.sort_values(by = 'order_requirement').tail(10)
#data_2005


,productCode,productLine,productName,quantityInStock,order_last_half_2005,order_requirement,suggestion
33,S18_2870,Classic Cars,1999 Indy 500 Monte Carlo SS,8164,136.888889,0.0,No
32,S18_2795,Vintage Cars,1928 Mercedes-Benz SSK,548,110.163934,0.0,No
31,S18_2625,Motorcycles,1936 Harley Davidson El Knucklehead,4357,152.962963,0.0,No
30,S18_2581,Planes,P-51-D Mustang,992,13.054054,0.0,No
29,S18_2432,Trucks and Buses,1926 Ford Fire Engine,2018,274.516129,0.0,No
39,S18_3232,Classic Cars,1992 Ferrari 360 Spider red,8347,296.734234,0.0,No
108,S72_3212,Ships,Pont Yacht,414,168.000000,0.0,No
84,S32_1374,Motorcycles,1997 BMW F650 ST,178,283.539568,106.0,Yes
6,S12_1099,Classic Cars,1968 Ford Mustang,68,259.000000,191.0,Yes
60,S24_2000,Motorcycles,1960 BSA Gold Star DBD34,15,257.985294,243.0,Yes


*CÂU 3: Chúng ta nên điều chỉnh các chiến lược tiếp thị và truyền thông như thế nào cho phù hợp với hành vi của khách hàng?*

In [ ]:
sql_q3_a ='''
--Summary financial & products data--

SELECT 	p.productCode,
			p.productLine,
			p.productName, 
			/*buyPrice, 
			min(priceEach) AS min_saleprice, 
			max(priceEach) AS max_saleprice,*/ 
			round(sum(quantityOrdered*priceEach)/sum(quantityOrdered),2) AS avg_saleprice,
			/*quantityInStock,*/
			SUM (quantityOrdered) AS total_order,
			SUM (quantityOrdered*priceEach) AS total_revenue,
			SUM ((priceEach-p.buyPrice)*quantityOrdered) as profit,
			ROUND(SUM((priceEach-p.buyPrice)*quantityOrdered)/sum(quantityOrdered),2) as profit_by_unit,
			ROUND(SUM((priceEach-p.buyPrice)*quantityOrdered)/SUM(quantityOrdered*priceEach)*100,2) as percent_profit,
			rank() over(order by SUM (quantityOrdered) DESC) as rank_order_qty,
			rank() over(order by sum(quantityOrdered*priceEach)/sum(quantityOrdered) DESC) as rank_avg_unit_price,
			rank() over(order by SUM (quantityOrdered*priceEach) DESC) as rank_revenue,
			rank() over(order by SUM ((priceEach-p.buyPrice)*quantityOrdered)/sum(quantityOrdered) DESC) as rank_profit_by_unit
	FROM 	orderdetails AS od
	JOIN 	products AS p
	  ON 	p.productCode = od.productCode
GROUP BY 	p.productCode
ORDER BY 	rank_profit_by_unit
LIMIT 15;'''

sql_q3_b ='''
--calculation of revenue by customers by years
WITH 
revenue_by_customers_2003 as(
	SELECT 	customerNumber, 
			sum(quantityOrdered)as total_order_2003,
			round(sum(quantityOrdered*priceEach)/sum(quantityOrdered),2) AS avg_saleprice_2003,
			round(SUM(quantityOrdered*priceEach),2) AS total_revenue_2003,
			rank() over(order by SUM(quantityOrdered*priceEach) DESC) as rank_2003												
	FROM 	orders AS o
	JOIN 	orderdetails AS od
	  ON 	o.orderNumber = od.orderNumber
   WHERE 	(orderDate >= '2003-01-01' AND orderDate <= '2003-12-31') 
GROUP BY 	customerNumber
ORDER BY 	total_revenue_2003 DESC
),

revenue_by_customers_2004 as(
	SELECT 	customerNumber, 
			sum(quantityOrdered) as total_order_2004,
			round(sum(quantityOrdered*priceEach)/sum(quantityOrdered),2) AS avg_saleprice_2004,
			round(SUM(quantityOrdered*priceEach),2) AS total_revenue_2004,
			rank() over(order by SUM(quantityOrdered*priceEach) DESC) as rank_2004
																		
	FROM 	orders AS o
	JOIN 	orderdetails AS od
	  ON 	o.orderNumber = od.orderNumber
   WHERE 	(orderDate >= '2004-01-01' AND orderDate <= '2004-12-31') 
GROUP BY 	customerNumber
ORDER BY 	total_revenue_2004 DESC
),

revenue_by_customers_2005 as(
	SELECT 	customerNumber, 
			sum(quantityOrdered) as Jan_May_order_2005,
			round(sum(quantityOrdered*priceEach)/sum(quantityOrdered),2) AS avg_saleprice_2005,
			round(SUM(quantityOrdered*priceEach),2) AS Jan_May_revenue_2005,
			rank() over(order by SUM(quantityOrdered*priceEach) DESC) as rank_2005
																		
	FROM 	orders AS o
	JOIN 	orderdetails AS od
	  ON 	o.orderNumber = od.orderNumber
   WHERE 	(orderDate >= '2005-01-01') 
GROUP BY 	customerNumber
ORDER BY 	Jan_May_revenue_2005 DESC
)
---Sum revenue by customers_Top 15 2005 compared to 2003-2004---
  SELECT	c.customerName, revenue_2005.customerNumber, revenue_2005.Jan_May_revenue_2005, revenue_2004.total_revenue_2004, revenue_2003.total_revenue_2003, revenue_2005.rank_2005, revenue_2004.rank_2004, revenue_2003.rank_2003
	FROM 	revenue_by_customers_2005 as revenue_2005
LEFT JOIN 	revenue_by_customers_2004 AS revenue_2004
	  ON 	revenue_2005.customerNumber = revenue_2004.customerNumber
LEFT JOIN 	revenue_by_customers_2003 AS revenue_2003
	  ON 	revenue_2005.customerNumber = revenue_2003.customerNumber
	  JOIN customers AS c
	  ON	revenue_2005.customerNumber = c.customerNumber
ORDER by	Jan_May_revenue_2005 DESC
   LIMIT	15;
'''

In [ ]:
qry = con.execute(sql_q3_a).fetchall()
a = pd.DataFrame(qry)
a = a.rename(columns={0: 'productCode', 1: 'productLine', 2: 'productName', 3: 'avg_saleprice', 4: 'total_order', 5: 'total_revenue', 6: 'profit', 7: 'profit_by_unit', 8: 'percent_profit', 9: 'rank_order_qty', 10: 'rank_avg_unit_price', 11: 'rank_revenue', 12: 'rank_profit_by_unit'})
a

,productCode,productLine,productName,avg_saleprice,total_order,total_revenue,profit,profit_by_unit,percent_profit,rank_order_qty,rank_avg_unit_price,rank_revenue,rank_profit_by_unit
0,S10_1949,Classic Cars,1952 Alpine Renault 1300,197.73,961,190017.96,95282.58,99.15,50.14,56,1,3,1
1,S12_1108,Classic Cars,2001 Ferrari Enzo,187.20,1019,190755.86,93349.65,91.61,48.94,20,2,2,2
2,S10_4698,Motorcycles,2003 Harley-Davidson Eagle Drag Bike,173.29,985,170686.00,81031.30,82.27,47.47,38,3,4,3
3,S12_1099,Classic Cars,1968 Ford Mustang,173.13,933,161531.48,72579.26,77.79,44.93,76,4,5,4
4,S18_2795,Vintage Cars,1928 Mercedes-Benz SSK,150.31,880,132275.98,68423.18,77.75,51.73,100,8,13,5
5,S18_3232,Classic Cars,1992 Ferrari 360 Spider red,153.12,1808,276839.98,135996.78,75.22,49.12,1,6,1,6
6,S12_3891,Classic Cars,1969 Ford Falcon,158.08,965,152543.02,72399.77,75.03,47.46,54,5,6,7
7,S18_1749,Vintage Cars,1917 Grand Touring Sedan,153.09,918,140535.60,60945.00,66.39,43.37,80,7,9,8
8,S18_3685,Classic Cars,1948 Porsche Type 356 Roadster,128.33,948,121653.46,62725.78,66.17,51.56,66,18,19,9
9,S12_2823,Motorcycles,2002 Suzuki XREO,132.07,1028,135767.03,67641.47,65.80,49.82,18,14,10,10


- *Nên tập trung nhiều vào dòng Vintage cars & Classic cars do lợi nhuận cao hơn các dòng sản phẩm khác & khách hàng cũng sẵn sàng trả giá cao cho những sản phẩm họ ưa thích (như S18_3232).*
  - *Should focus more on Vintage cars & Classic cars due to their higher profits than other Product lines & customers are also willing to pay high price for their preferred products (like S18_3232).*
- *Sản phẩm mang lại hiệu quả cao nhất chủ yếu là ở mức giá cực cao (mặt hàng được liệt kê từ thứ 1 đến thứ 4), xem xét giảm giá hoặc giảm giá nhiều hơn cho những sản phẩm này hoặc chi nhiều hơn để thu hút sự quan tâm của khách hàng đối với những mặt hàng cao cấp đó.*
  - *Product bringing highest products mainly at the extremely high price (1st to 4th listed items), considering to reduce the price or offer more discount on these products or spending more for attracting customer’s interest on such premium items.*
- *Các mặt hàng ở mức giá trung bình và mang lại lợi nhuận tốt nhưng được đặt hàng với tỷ lệ rất thấp -> nên chi nhiều hơn cho các hoạt động tiếp thị này để tìm kiếm khách hàng mới hoặc tổ chức nhiều chiến dịch để giới thiệu nhiều hơn các sản phẩm này ra thị trường. Cũng đang xem xét chiết khấu thêm cho khách hàng lớn để đẩy doanh số (% lợi nhuận không cần quá cao như hiện tại).*
  - *Items at the average price and bringing good profit but being ordered at very low rate -> should spending more on these marketing to find new customers or organize more campaigns to introduce more of these products to the market. Also considering discount more for big customers to boost their sales (%profit not need to be so much high as current).*

In [ ]:
qry = con.execute(sql_q3_b).fetchall()
b = pd.DataFrame(qry).fillna(0)
b = b.rename(columns={0: 'customerName', 1: 'customerNumber', 2: 'Jan_May_revenue_2005', 3: 'total_revenue_2004', 4: 'total_revenue_2003', 5: 'rank_2005', 6: 'rank_2004', 7: 'rank_2003'})
b.rank_2003 = b.rank_2003.astype('int64')
b.rank_2004 = b.rank_2004.astype('int64')
b

,customerName,customerNumber,Jan_May_revenue_2005,total_revenue_2004,total_revenue_2003,rank_2005,rank_2004,rank_2003
0,Euro+ Shopping Channel,141,290018.52,340830.87,189840.15,1,1,1
1,Mini Gifts Distributors Ltd.,124,192481.73,231562.53,167783.08,2,2,2
2,La Rochelle Gifts,119,91147.11,67426.01,0.00,3,18,0
3,The Sharp Gifts Warehouse,450,83984.89,59551.38,0.00,4,26,0
4,"Down Under Souveniers, Inc",323,75020.13,76721.95,2880.00,5,13,73
5,"Anna's Decorations, Ltd",276,56932.30,0.00,80101.92,6,0,10
6,Salzburg Collectables,382,52420.07,49233.67,35826.33,7,41,40
7,Gifts4AllAges.com,362,50806.85,33533.47,0.00,8,56,0
8,Corporate Gift Ideas Co.,321,46781.66,0.00,85559.12,9,0,8
9,"Oulu Toy Supplies, Inc.",311,46770.52,16212.59,32723.04,10,73,46


- *Top 2 customers (#141 & #124) has been kept unchanged over years with high contribution to the total income of the Company.*
  - *Top 2 khách hàng (#141 & #124) được giữ nguyên qua các năm và đóng góp cao vào tổng thu nhập của Công ty.*
- *10th-14th ranking customers has been working with the Company since 2003 without very outstanding purchased volume, but this is reflecting their high loyalty and interest in the Company’s products.*
  - *Các khách hàng xếp hạng 10-14 đã làm việc với Công ty từ năm 2003 với lượng mua không quá nổi bật, nhưng điều này thể hiện sự trung thành và quan tâm cao của họ đối với các sản phẩm của Công ty.*
- *Maintain good relationship to retain top 2 customers (#141 & #124) due to their high loyalty and profit contribution to the Company. Considering offer better selling policy for these customers for rewarding and boosting revenue from them (higher discount on products, introduce new products which may attract their attention and increase their demand.*
  - *Duy trì mối quan hệ tốt để giữ chân 2 khách hàng hàng đầu (#141 & #124) do họ có mức độ trung thành cao và đóng góp lợi nhuận cho Công ty. Xem xét đưa ra chính sách bán hàng tốt hơn cho những khách hàng này để thưởng và tăng doanh thu từ họ (chiết khấu cao hơn cho sản phẩm, giới thiệu sản phẩm mới có thể thu hút sự chú ý của họ và tăng nhu cầu của họ.*
- *Pay more attention on loyal customers which have the good and continuous business with the Company since 2003, offer better selling policy to encourage their higher purchase of products as well as improving competitive advantage in the market.*
  - *Quan tâm hơn nữa đến các khách hàng thân thiết có quan hệ làm ăn tốt và liên tục với Công ty từ năm 2003, đưa ra các chính sách bán hàng tốt hơn để khuyến khích họ mua sản phẩm nhiều hơn cũng như nâng cao lợi thế cạnh tranh trên thị trường.*
- *Focus more on Sales & Marketing areas to keep the good relationship with customers, understanding the reason of their unstable purchase over years (due to unstable demand or better offer from other suppliers) for building a solution for competing with others in the market, building a continuous growth, better planning and forecasting sales/inventories/ costs in the future.*
  - *Tập trung nhiều hơn vào mảng Sales & Marketing để giữ mối quan hệ tốt với khách hàng, tìm hiểu nguyên nhân mua hàng không ổn định của họ qua các năm (do nhu cầu không ổn định hoặc do nhà cung cấp khác chào hàng tốt hơn) để xây dựng giải pháp cạnh tranh với các đối thủ khác trên thị trường, xây dựng tăng trưởng liên tục, lập kế hoạch tốt hơn và dự báo doanh thu/hàng tồn kho/chi phí trong tương lai.*

*CÂU 4: Chúng ta có thể chi bao nhiêu để có được khách hàng mới?*
- *Để tính CAC (Customer Acquisition Cost) chi phí để có một khách hàng mới mà vẫn có lợi nhuận.*
    - *How to calculate CAC (Customer Acquisition Cost) to get a new customer and still make a profit.*
- *Nếu chi nhiều hơn CAC thì sẽ mất tiền và bị lỗ, nếu chi ít hơn thì sẽ giới hạn khả năng phát triển và mở rộng của doanh nghiệp.*
    - *If spending more than CAC, you will lose money and make errors. On the contrary, it will limit your ability to grow and expand.*

In [13]:
# B1: Tính LVC (giá trị trọn đời của khách hàng) [Tổng doanh thu / số lượng khách hàng (trong cùng 1 chu kỳ)]
period = datamix[(datamix['orderDate'] >= '2004-01-01') & (datamix['orderDate'] <= '2004-12-31') & (datamix['status'] != 'Cancelled')]
LVC = round((period['quantityOrdered'] * period['priceEach']).sum() / period['customerNumber'].nunique(),2)
# B2: Chi phí hoàn tiền / chi phí huỷ đơn hàng
period_full = datamix[(datamix['orderDate'] >= '2004-01-01') & (datamix['orderDate'] <= '2004-12-31')]
shipped_ratio = round(period['status'].count() / period_full['status'].count(), 2)
cancel_cost = round(LVC * (1-shipped_ratio), 2)
# B3: Chi phí cho đơn hàng bán được (đã tính vào giá bán)
# B4: Chi phí cho việc vận hành hằng ngày (đã tính vào giá bán)
# B5: Lợi nhuận trên mỗi khách hàng mà bạn muốn có
profit = ((period['priceEach'] - period['buyPrice']) * period['quantityOrdered']).sum()
sales = (period['priceEach'] * period['quantityOrdered']).sum()
profit_margin = profit / sales 
profit_per_customer = LVC * profit_margin

CAC = round((LVC - cancel_cost - profit_per_customer), 2)
CAC

27940.69

In [11]:
sql_q4 = """ SELECT ROUND(
(SELECT ROUND((SUM(quantityOrdered * priceEach) / COUNT(DISTINCT customerNumber)),2)
FROM orders AS o
  JOIN orderdetails AS od
       ON o.orderNumber = od.orderNumber
  JOIN products AS p
      ON od.productCode = p.productCode
WHERE orderDate BETWEEN '2004-01-01' AND '2004-12-31' AND status NOT LIKE 'Cancelled')
* 
( 1 - 
(SELECT (SELECT COUNT (status) AS num_cancelled_order
FROM orders AS o
  JOIN orderdetails AS od
       ON o.orderNumber = od.orderNumber
  JOIN products AS p
      ON od.productCode = p.productCode
WHERE orderDate BETWEEN '2004-01-01' AND '2004-12-31' AND status LIKE 'Cancelled')
*1.00
/
(SELECT COUNT (status) AS num_order
FROM orders AS o
  JOIN orderdetails AS od
       ON o.orderNumber = od.orderNumber
  JOIN products AS p
      ON od.productCode = p.productCode
WHERE orderDate BETWEEN '2004-01-01' AND '2004-12-31') )
-
( SELECT  CAST(ROUND((SUM(quantityOrdered*priceEach) - SUM(quantityOrdered*buyPrice)) / SUM(quantityOrdered*priceEach),6) AS VARCHAR)  AS Profit_Margin
				FROM products AS p
				JOIN orderdetails AS od
				ON p.productCode = od.productCode
				JOIN orders AS o
				ON od.orderNumber = o.orderNumber
				WHERE orderDate BETWEEN '2004-01-01' AND '2004-12-31' AND status NOT LIKE 'Cancelled')
),2) AS CAC 
"""

In [12]:
qry = con.execute(sql_q4).fetchall()
answer4 = pd.DataFrame(qry)
answer4 = answer4.rename(columns={0:'CAC'})
answer4

,CAC
0,28040.48
